In [8]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import sys
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()

In [9]:
print os.getcwd();

c:\Users


In [11]:
# if necessary change the directory
#os.chdir('c:\\Users\..')

In [12]:
data = pd.read_csv("nightlife_sanfrancisco_en.csv", header=0, delimiter=",")

In [13]:
# iexplore data set
data.shape

(34086, 3)

In [14]:
data.columns.values

array(['company', 'text', 'stars'], dtype=object)

In [15]:
print data["text"][0]

Great experience with Joey for a family karaoke night! He had every song that was requested by attendees. He literally had over 50,000+ songs to choose from -- everything from top 40s, R&B, hip hop, country, pop, and every song from any and every decade. He also played great dance music during transitions between singers.  

He was extremely professional and accommodating throughout the entire planning process and during the actual event. We will definitely be using Joey again for future events - I highly recommend him for any of his services!



In [16]:
# Remove stop words from "words"
import nltk      # import stop words
nltk.download('popular')  # Download text data sets, including stop words

[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Magda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Magda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Magda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Magda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Magda\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]  

True

In [17]:
from nltk.corpus import stopwords # Import the stop word list
print stopwords.words("english")
#words = [w for w in words if not w in stopwords.words("english")]
#print words #  "u" before each word indicates that Python is internally representing each word as a unicode string

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [19]:
# Clean all records
def text_to_words( raw_text ):
    # 1. Remove end of line
    without_end_line = re.sub('\n', ' ', raw_text)
    # 2. Remove start of line
    without_start_line = re.sub('\r', ' ', without_end_line)
    # 3. Remove punctuation
    without_punctual = re.sub(ur'[\W_]+',' ',without_start_line )
    # 4. Replace number by XnumberX
    without_number = re.sub('(\d+\s*)+', ' XnumberX ', without_punctual)
    # 5. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", without_number) 
    # 6. Convert to lower case
    lower_case = letters_only.lower()
    # 7. Split into individual words
    words = lower_case.split()
    # 8. stemming  - algorithms Porter stemmer
    meaningful_words = [ps.stem(word) for word in words]
    # 9.Remove stop words 
    # Redundant step, removing later in Creating the bag of words step
    #stops = set(stopwords.words("english"))                  
    #meaningful_words = [w for w in words if not w in stops]   
    # 10. Join the words back into one string separated by space and return the result.
    return( " ".join( meaningful_words ))
    #return (meaningful_words)

In [20]:
clean_text = text_to_words( data["text"][0] )
print clean_text

great experi with joey for a famili karaok night he had everi song that wa request by attende he liter had over xnumberx song to choos from everyth from top xnumberx s r b hip hop countri pop and everi song from ani and everi decad he also play great danc music dure transit between singer he wa extrem profession and accommod throughout the entir plan process and dure the actual event we will definit be use joey again for futur event i highli recommend him for ani of hi servic


In [21]:
# Get the number of text based on the dataframe column size
num_text = data["text"].size
# Initialize an empty list to hold the clean text
clean_data = []
# Loop over each text; create an index i that goes from 0 to the length
print "Cleaning and parsing the data set text...\n"
clean_data = []
for i in xrange( 0, num_text ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Text %d of %d\n" % ( i+1, num_text )                                                                    
    clean_data.append( text_to_words( data["text"][i] )) # in case of error run "pip install -U nltk"

Cleaning and parsing the data set text...

Text 1000 of 34086

Text 2000 of 34086

Text 3000 of 34086

Text 4000 of 34086

Text 5000 of 34086

Text 6000 of 34086

Text 7000 of 34086

Text 8000 of 34086

Text 9000 of 34086

Text 10000 of 34086

Text 11000 of 34086

Text 12000 of 34086

Text 13000 of 34086

Text 14000 of 34086

Text 15000 of 34086

Text 16000 of 34086

Text 17000 of 34086

Text 18000 of 34086

Text 19000 of 34086

Text 20000 of 34086

Text 21000 of 34086

Text 22000 of 34086

Text 23000 of 34086

Text 24000 of 34086

Text 25000 of 34086

Text 26000 of 34086

Text 27000 of 34086

Text 28000 of 34086

Text 29000 of 34086

Text 30000 of 34086

Text 31000 of 34086

Text 32000 of 34086

Text 33000 of 34086

Text 34000 of 34086



In [22]:
# Compare original and edited text
data['text'][0]

'Great experience with Joey for a family karaoke night! He had every song that was requested by attendees. He literally had over 50,000+ songs to choose from -- everything from top 40s, R&B, hip hop, country, pop, and every song from any and every decade. He also played great dance music during transitions between singers.  \r\n\r\nHe was extremely professional and accommodating throughout the entire planning process and during the actual event. We will definitely be using Joey again for future events - I highly recommend him for any of his services!\r\n'

In [23]:
clean_data[0]

u'great experi with joey for a famili karaok night he had everi song that wa request by attende he liter had over xnumberx song to choos from everyth from top xnumberx s r b hip hop countri pop and everi song from ani and everi decad he also play great danc music dure transit between singer he wa extrem profession and accommod throughout the entir plan process and dure the actual event we will definit be use joey again for futur event i highli recommend him for ani of hi servic'

In [24]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = 'english',   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_data)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [25]:
print train_data_features.shape

(34086L, 5000L)


In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

In [ ]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

In [96]:
# Using in model, random forest example
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["stars"] )

Training the random forest...
